In [1]:
from typing import Dict, Optional, Any, List, Self, Literal, ClassVar, Union
from uuid import UUID, uuid4
from pydantic import BaseModel, Field, model_validator, computed_field, field_validator
from enum import Enum
from dnd.values import ModifiableValue, AdvantageStatus, CriticalStatus, AutoHitStatus, StaticValue,NumericalModifier, ContextualValue

from dnd.blocks import (
    BaseBlock, AbilityScores, SavingThrowSet, Health, 
    Equipped, Speed, ActionEconomy,SkillSet,SkillName
)
from dnd.values import ModifiableValue
from dnd.modifiers import (
    NumericalModifier, DamageType, ResistanceStatus, 
    ContextAwareCondition, BaseObject
)

from dnd.equipment import (
    Armor, Weapon, Shield, BodyArmor, Gauntlets, Greaves,
    Boots, Amulet, Ring, Cloak, Helmet, BodyPart
)
from dnd.dice import Dice, RollType, DiceRoll

In [2]:
from dnd.modifiers import (
    BaseObject,
    naming_callable,
    NumericalModifier,
    AdvantageModifier,
    CriticalModifier,
    AutoHitModifier,
    AdvantageStatus,
    CriticalStatus,
    AutoHitStatus,
    Size,
    DamageType,
    SizeModifier,
    DamageTypeModifier,
    ContextualNumericalModifier,
    ContextualAdvantageModifier,
    ContextualCriticalModifier,
    ContextualAutoHitModifier,
    ContextualSizeModifier,
    ContextualDamageTypeModifier,
    ResistanceModifier,
    ContextualResistanceModifier,
    ResistanceStatus
)
from dnd.entity import Entity

In [3]:
#static advantage modifier


In [4]:
goblin = Entity.create(source_entity_uuid=uuid4(),name="Goblin")
skeleton = Entity.create(source_entity_uuid=uuid4(),name="Skeleton")
print(goblin.source_entity_uuid)
print(goblin.uuid)
print(skeleton.source_entity_uuid)
print(skeleton.uuid)
goblin.set_target_entity(goblin.uuid)


852c0f7a-bd8c-4ca7-8df5-036083f2f676
852c0f7a-bd8c-4ca7-8df5-036083f2f676
3f9a9da8-95bc-47c7-a625-4adb51965cdb
3f9a9da8-95bc-47c7-a625-4adb51965cdb


In [5]:
def get_bonuses_for_skill(entity: Entity, skill_name: SkillName):
    proficiency_bonus = entity.proficiency_bonus
    skill = entity.skill_set.get_skill(skill_name)
    skill_bonus = skill.skill_bonus
    proficiency_bonus_multiplier_callable = skill._get_proficiency_converter()
    ability_name = skill.ability
    ability = entity.ability_scores.get_ability(ability_name)
    ability_bonus = ability.ability_score
    ability_modifier_bonus = ability.modifier_bonus
    normalized_proficiency_bonus = proficiency_bonus.model_copy(deep=True)
    normalized_proficiency_bonus.update_normalizers(proficiency_bonus_multiplier_callable)
    return normalized_proficiency_bonus, skill_bonus, ability_bonus, ability_modifier_bonus

def skill_total_bonus(entity: Entity, skill_name: SkillName):
    """ For a skill roll we need to get:
     -  proficiency bonu from Entity
     -  skill bonus from Entity.skill_set.get_skill(skill_name).skill_bonus
     -  proficiency bonus multiplier from Entity.skill_set.get_skill(skill_name).proficiency_bonus_multiplier
     - ability bonus from Entity.ability_scores.get_ability(skill.ability)"""
    normalized_proficiency_bonus, skill_bonus, ability_bonus, ability_modifier_bonus = get_bonuses_for_skill(entity, skill_name)
    total_bonus = normalized_proficiency_bonus.combine_values([skill_bonus, ability_bonus, ability_modifier_bonus])
    return total_bonus




In [6]:
acrobatics_bonus = skill_total_bonus(goblin, "acrobatics")
print(acrobatics_bonus.normalized_score)
print(acrobatics_bonus.score)


Validating others source id 852c0f7a-bd8c-4ca7-8df5-036083f2f676 against self source 852c0f7a-bd8c-4ca7-8df5-036083f2f676 for modifer  with name proficiency_bonus and uuid d78ba75b-1301-4629-a74d-6a3298fe455e and type <class 'dnd.values.ModifiableValue'> against <class 'dnd.values.ModifiableValue'> with name Skill Bonus and uuid 96f8f3d6-4193-4fae-b11c-ee0b16827c27
Validating others source id 852c0f7a-bd8c-4ca7-8df5-036083f2f676 against self source 852c0f7a-bd8c-4ca7-8df5-036083f2f676 for modifer  with name proficiency_bonus and uuid d78ba75b-1301-4629-a74d-6a3298fe455e and type <class 'dnd.values.ModifiableValue'> against <class 'dnd.values.ModifiableValue'> with name Ability Score and uuid b8e9a08f-6c87-4a93-8b52-bd150be3ff73
Validating others source id 852c0f7a-bd8c-4ca7-8df5-036083f2f676 against self source 852c0f7a-bd8c-4ca7-8df5-036083f2f676 for modifer  with name proficiency_bonus and uuid d78ba75b-1301-4629-a74d-6a3298fe455e and type <class 'dnd.values.ModifiableValue'> against

In [7]:
a

NameError: name 'a' is not defined

In [6]:
from dnd.modifiers import ContextualModifier


self_advantage = AdvantageModifier(name="self_advantage",source_entity_uuid=goblin.uuid,target_entity_uuid=goblin.uuid,value=AdvantageStatus.ADVANTAGE)


def fear_of_skeleton(source_entity: UUID, target_entity: Optional[UUID], context: Optional[Dict[str, Any]] = None) -> AdvantageModifier:
    nonemodifier= AdvantageModifier(name="fear_of_skeleton",source_entity_uuid=source_entity,target_entity_uuid=target_entity,value=AdvantageStatus.NONE)
    if target_entity is None:
        return nonemodifier

    target_entity_name = Entity.get(target_entity)
    if target_entity_name is None:
        return nonemodifier
    if target_entity_name.name == "Skeleton":
        return AdvantageModifier(name="fear_of_skeleton",source_entity_uuid=source_entity,target_entity_uuid=target_entity,value=AdvantageStatus.DISADVANTAGE)
    else:


        return nonemodifier


ifskeleton_modifier= ContextualAdvantageModifier(name="contextual_fear_of_skeleton",source_entity_uuid=goblin.uuid,target_entity_uuid=goblin.uuid,callable=fear_of_skeleton)

negative_bonus_modifier = NumericalModifier(name="negative_bonus",source_entity_uuid=skeleton.uuid,target_entity_uuid=goblin.uuid,value=-4)

# from_target_static = StaticValue(name="from_target_static",source_entity_uuid=skeleton.uuid,target_entity_uuid=goblin.uuid, is_outgoing_modifier=True)
# from_target_static.add_value_modifier(negative_bonus_modifier)

def fear_of_goblin(source_entity: UUID, target_entity: Optional[UUID], context: Optional[Dict[str, Any]] = None) -> NumericalModifier:
    zero_modifier = NumericalModifier(name="zero_modifier",source_entity_uuid=source_entity,target_entity_uuid=target_entity,value=0)

    if target_entity is None:
        return zero_modifier
    target_entity_object = Entity.get(target_entity)
    if target_entity_object is None:
        return zero_modifier
    if target_entity_object.name == "Goblin":
        return NumericalModifier(name="fear_of_goblin",source_entity_uuid=source_entity,target_entity_uuid=target_entity,value=6)
    else:
        return zero_modifier
    
fear_of_goblin_contextual_modifier = ContextualNumericalModifier(name="fear_of_goblin",source_entity_uuid=skeleton.uuid, target_entity_uuid=skeleton.uuid,callable=fear_of_goblin)



In [7]:
#try to compute the modifieable value for a skill roll
# gotta get a) the proficiency bonus b) the ability modifier c) the skill bonus
# we want to keep them as modifiable value and combine them together
assert isinstance(goblin, Entity)
proficiency_modifier = goblin.proficiency_bonus
extracted_skill = goblin.skill_set.get_skill("acrobatics")
skill_modifier = extracted_skill.skill_bonus
extracted_ability = goblin.ability_scores.get_ability(extracted_skill.ability)
ability_modifier = extracted_ability.ability_score
#oprint all source entity uuids
print(f"Proficiency Bonus: {proficiency_modifier.source_entity_uuid}")
print(f"Skill Bonus: {skill_modifier.source_entity_uuid}")
print(f"Ability Modifier: {ability_modifier.source_entity_uuid}")


Proficiency Bonus: 638a7731-5377-4e61-b73d-c08f8a537a5b
Skill Bonus: 638a7731-5377-4e61-b73d-c08f8a537a5b
Ability Modifier: 638a7731-5377-4e61-b73d-c08f8a537a5b


In [8]:
skeleton_proficiency_modifier = skeleton.proficiency_bonus
skeleton_extracted_skill = skeleton.skill_set.get_skill("acrobatics")
skeleton_skill_modifier = skeleton_extracted_skill.skill_bonus
skeleton_extracted_ability = skeleton.ability_scores.get_ability(skeleton_extracted_skill.ability)
skeleton_ability_modifier = skeleton_extracted_ability.ability_score


skelton_to_target_static = skeleton_ability_modifier.to_target_static
skeleton_to_target_contextual = skeleton_ability_modifier.to_target_contextual
skeleton_to_target_contextual.add_value_modifier(fear_of_goblin_contextual_modifier)
skelton_to_target_static.add_value_modifier(negative_bonus_modifier)
skeleton.set_target_entity(goblin.uuid)
goblin.set_target_entity(skeleton.uuid)

In [9]:
print(ability_modifier.normalized_score)
print(ability_modifier.score)
print(skill_modifier.normalized_score)
print(skill_modifier.score)
combined_modifier= ability_modifier.combine_values([skill_modifier])
print(combined_modifier.normalized_score)
print(combined_modifier.score)


0
10
0
0
Validating others source id 638a7731-5377-4e61-b73d-c08f8a537a5b against self source 638a7731-5377-4e61-b73d-c08f8a537a5b for modifer  with name Ability Score and uuid 29f1c9e6-d3c6-4c31-a670-056e081eb639 and type <class 'dnd.values.ModifiableValue'> against <class 'dnd.values.ModifiableValue'> with name Skill Bonus and uuid b850e759-c54d-4c0b-b148-eeaa9e354dce
0
10


In [10]:
ability_modifier.set_from_target_static(skelton_to_target_static)
ability_modifier.set_from_target_contextual(skeleton_to_target_contextual)


In [11]:
print(ability_modifier.normalized_score)
print(ability_modifier.score)
print(skill_modifier.normalized_score)
print(skill_modifier.score)


2
12
0
0


In [12]:
goblin.name="GOBLINO"
print(ability_modifier.normalized_score)
print(ability_modifier.score)
print(skill_modifier.normalized_score)
print(skill_modifier.score)
goblin.name="Goblin"

-4
6
0
0


In [13]:
combined_modifier2= ability_modifier.combine_values([skill_modifier])
print(combined_modifier2.normalized_score)
print(combined_modifier2.score)

Validating others source id 638a7731-5377-4e61-b73d-c08f8a537a5b against self source 638a7731-5377-4e61-b73d-c08f8a537a5b for modifer  with name Ability Score and uuid 29f1c9e6-d3c6-4c31-a670-056e081eb639 and type <class 'dnd.values.ModifiableValue'> against <class 'dnd.values.ModifiableValue'> with name Skill Bonus and uuid b850e759-c54d-4c0b-b148-eeaa9e354dce
2
12


In [14]:
combined_modifier2.get_typed_modifiers()


[StaticValue(name='A Value_A Value', uuid=UUID('42c2247d-f7e2-41cb-b2cd-13b84e07166e'), source_entity_uuid=UUID('638a7731-5377-4e61-b73d-c08f8a537a5b'), source_entity_name=None, target_entity_uuid=UUID('366a13c5-1c36-4b39-8d63-4e1ea4136b94'), target_entity_name=None, context=None, score_normalizer=<function ability_score_normalizer at 0x1071ced40>, generated_from=[UUID('89582156-63b8-4566-ad6e-cc93f25d2e89'), UUID('4fa354bd-3fde-4dc8-9e31-738b15021277')], global_normalizer=False, value_modifiers={UUID('f62a7876-aac9-4a9b-8744-6bacda02ca89'): NumericalModifier(name='Ability Score_base_value', uuid=UUID('f62a7876-aac9-4a9b-8744-6bacda02ca89'), source_entity_uuid=UUID('3cc07c31-9f94-4764-82ff-2560016e5706'), source_entity_name=None, target_entity_uuid=UUID('3cc07c31-9f94-4764-82ff-2560016e5706'), target_entity_name=None, context=None, value=10, score_normalizer=<function ability_score_normalizer at 0x1071ced40>, normalized_value=0), UUID('5f3d3082-19fe-4681-9b79-884dda097a94'): NumericalM

In [15]:
combined_modifier2.clear_target_entity()
print(combined_modifier2.normalized_score)
print(combined_modifier2.score)

0
10


In [29]:
combined_modifier= ability_modifier.combine_values([skill_modifier])
print(combined_modifier.normalized_score)
print(combined_modifier.score)

Validating others source id 6a8b6d91-4179-40e7-ba74-06faf69e301d against self source 6a8b6d91-4179-40e7-ba74-06faf69e301d for modifer  with name Ability Score and uuid c58d9aca-4386-4fd1-94e8-32fb31b6c14a and type <class 'dnd.values.ModifiableValue'> against <class 'dnd.values.ModifiableValue'> with name Skill Bonus and uuid bfb97730-32ac-4ed1-a1cc-55c8fd009630
0
10


In [30]:
combined_modifier.get_generated_from()

[ModifiableValue(name='Ability Score', uuid=UUID('c58d9aca-4386-4fd1-94e8-32fb31b6c14a'), source_entity_uuid=UUID('6a8b6d91-4179-40e7-ba74-06faf69e301d'), source_entity_name=None, target_entity_uuid=UUID('6a8b6d91-4179-40e7-ba74-06faf69e301d'), target_entity_name=None, context=None, score_normalizer=<function ability_score_normalizer at 0x105ed2ca0>, generated_from=[], global_normalizer=True, self_static=StaticValue(name='A Value', uuid=UUID('eca6bf0f-435a-4b3c-babb-87a79cd30233'), source_entity_uuid=UUID('6a8b6d91-4179-40e7-ba74-06faf69e301d'), source_entity_name=None, target_entity_uuid=UUID('6a8b6d91-4179-40e7-ba74-06faf69e301d'), target_entity_name=None, context=None, score_normalizer=<function ability_score_normalizer at 0x105ed2ca0>, generated_from=[], global_normalizer=True, value_modifiers={UUID('59ac02b8-c23f-49b7-890f-a3f7354f35f5'): NumericalModifier(name='Ability Score_base_value', uuid=UUID('59ac02b8-c23f-49b7-890f-a3f7354f35f5'), source_entity_uuid=UUID('db54ae10-a097-493

In [31]:
ability_modifier.global_normalizer

True

In [32]:
ability_modifier.score_normalizer

<function dnd.blocks.ability_score_normalizer(score: int) -> int>

In [33]:
numerical_modifier_dict = ability_modifier.self_static.value_modifiers
numerical_modifier = numerical_modifier_dict[list(ability_modifier.self_static.value_modifiers.keys())[0]]

In [34]:
print(numerical_modifier.value)
print(numerical_modifier.normalized_value)
print(numerical_modifier.score_normalizer)

10
0
<function ability_score_normalizer at 0x105ed2ca0>


In [35]:
numerical_modifier = ability_modifier.self_static.value_modifiers[list(ability_modifier.self_static.value_modifiers.keys())[0]]



In [36]:
normalizer =numerical_modifier.score_normalizer
normalizer

<function dnd.blocks.ability_score_normalizer(score: int) -> int>

In [37]:
normalizer(19)

4

In [38]:
proficiency_modifier.self_static.target_entity_uuid


UUID('6a8b6d91-4179-40e7-ba74-06faf69e301d')

In [39]:
skill_modifier.self_static.target_entity_uuid

UUID('6a8b6d91-4179-40e7-ba74-06faf69e301d')

In [40]:
from typing import Tuple
def skill_merge_status(proficiency_bonus: ModifiableValue, skill_bonus: ModifiableValue, ability_modifier: ModifiableValue) -> Tuple[AdvantageStatus, CriticalStatus, AutoHitStatus, int, int]:
    merged_value = proficiency_bonus.combine_values([skill_bonus, ability_modifier])          

    return merged_value.advantage,merged_value.critical,merged_value.auto_hit,merged_value.normalized_score,merged_value.score
goblin.set_target_entity(goblin.uuid)

advantage_status = skill_merge_status(proficiency_modifier, skill_modifier, ability_modifier)
print(advantage_status)
proficiency_modifier.self_static.add_advantage_modifier(self_advantage)
proficiency_modifier.self_contextual.add_advantage_modifier(ifskeleton_modifier)
advantage_status_post = skill_merge_status(proficiency_modifier, skill_modifier, ability_modifier)
print(advantage_status_post)
goblin.set_target_entity(skeleton.uuid)
advantage_status_post_target = skill_merge_status(proficiency_modifier, skill_modifier, ability_modifier)
print(advantage_status_post_target)




Validating others source id 6a8b6d91-4179-40e7-ba74-06faf69e301d against self source 6a8b6d91-4179-40e7-ba74-06faf69e301d for modifer  with name proficiency_bonus and uuid ef383dfc-3725-4180-a062-88165f3b96ae and type <class 'dnd.values.ModifiableValue'> against <class 'dnd.values.ModifiableValue'> with name Skill Bonus and uuid bfb97730-32ac-4ed1-a1cc-55c8fd009630
Validating others source id 6a8b6d91-4179-40e7-ba74-06faf69e301d against self source 6a8b6d91-4179-40e7-ba74-06faf69e301d for modifer  with name proficiency_bonus and uuid ef383dfc-3725-4180-a062-88165f3b96ae and type <class 'dnd.values.ModifiableValue'> against <class 'dnd.values.ModifiableValue'> with name Ability Score and uuid c58d9aca-4386-4fd1-94e8-32fb31b6c14a
(<AdvantageStatus.NONE: 'None'>, <CriticalStatus.NONE: 'None'>, <AutoHitStatus.NONE: 'None'>, 2, 12)
Validating others source id 6a8b6d91-4179-40e7-ba74-06faf69e301d against self source 6a8b6d91-4179-40e7-ba74-06faf69e301d for modifer  with name proficiency_bonu

RecursionError: maximum recursion depth exceeded

In [ ]:
ifskeleton_modifier.uuid
proficiency_modifier.remove_modifier(ifskeleton_modifier.uuid)
advantage_status_post_remove = skill_merge_status(proficiency_modifier, skill_modifier, ability_modifier)
print(advantage_status_post_remove)

In [ ]:
#formatetd print of the bonus
print(f"Proficiency Bonus: {proficiency_modifier.source_entity_uuid}")
print(f"Skill Bonus: {skill_modifier.source_entity_uuid}")
print(f"Ability Modifier: {ability_modifier.source_entity_uuid}")



In [ ]:
proficiency_modifier.normalized_score

In [ ]:
from typing import Tuple, Optional
import random

def roll_d20(advantage_status: AdvantageStatus) -> Tuple[int,Optional[int]]:
        num_rolls = 1 if advantage_status == AdvantageStatus.NONE else 2
        rolls = [random.randint(1,20) for _ in range(num_rolls)]
        if advantage_status == AdvantageStatus.ADVANTAGE:
            return max(rolls), min(rolls)
        elif advantage_status == AdvantageStatus.DISADVANTAGE:
            return min(rolls), max(rolls)
        else:
            return rolls[0], None
        
        
def skill_roll_bonus(skill_name: SkillName, entity: Entity) -> DiceRoll:
    extracted_skill = entity.skill_set.get_skill(skill_name)
    skill_modifier = extracted_skill.skill_bonus
    extracted_ability = entity.ability_scores.get_ability(extracted_skill.ability)
    ability_modifier = extracted_ability.ability_score



        

       
    proficiency_modifier = entity.proficiency_bonus

    

    return DiceRoll(
        roll_type=RollType.SKILL,
        results=,
        source_entity_uuid=entity.uuid,
        target_entity_uuid=None,
        attack_outcome=None
    )

